### Building on top of these concepts:

If you haven't covered langmem and basic openai-agents-sdk go through these notebooks first:

1. [Understand LangMem Core APIs](https://colab.research.google.com/drive/1YJNrnQRMgeNTigIuWOfykt-Z5L_DDmsa?usp=sharing)

2. [Use LangMem Memory Tools with OpenAI Agents SDK](https://colab.research.google.com/drive/1xgSUeJPIBKyjpM868PsvmCZaCof-s2vB?usp=sharing)

3. [Use Persistence Storage](https://colab.research.google.com/drive/1gA9r_FkbHFCWlgd52qz1oTW736gD0kmN?usp=sharing)

4. [Baseline Email Assistant - Prep before Memory](https://colab.research.google.com/drive/1AgedinzRuoow3f2cvR0wRlATk8AgE0x0?usp=sharing)

5. [OpenAI Agents SDK Email Assistant with Semantic](https://colab.research.google.com/drive/1L4T8eZIHzYD1OQtC8hubZMeFy-eRmxV2?usp=sharing)

6. [OpenAI Agents SDK Email Assistant with Semantic & Episodic Memories](https://colab.research.google.com/drive/1pdo1COE3gxo2LPUYLm92O-o_1xFJKQyq?usp=sharing)

## Install Packages

In [1]:
!pip install -Uq openai-agents langmem langchain-google-genai langmem_adapter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")

# **Procedural Memory**

Procedural memory encodes how an agent should behave and respond. It starts with system prompts that define core behavior, then evolves through feedback and experience. As the agent interacts with users, it refines these instructions, learning which approaches work best for different situations.

# 1. **Procedural Memory: System Instructions**

https://langchain-ai.github.io/langmem/reference/prompt_optimization/

In [4]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer(
    "google_genai:gemini-2.0-flash",
    kind="metaprompt",
    config={"max_reflection_steps": 3}
)

In [5]:
prompt = "You are a helpful assistant."
trajectory = [
    {"role": "user", "content": "Explain inheritance in Python"},
    {"role": "assistant", "content": "Here's a detailed theoretical explanation..."},
    {"role": "user", "content": "Show me a practical example instead"},
]
optimized = optimizer.invoke({
    "trajectories": [(trajectory, {"user_score": 0})],
    "prompt": prompt
})
print(optimized)

You are a helpful assistant. When explaining programming concepts, always start with a practical example (e.g., a code snippet) before providing any theoretical explanation. If the user asks for a theoretical explanation, then provide it after the practical example.


# **OpenAI Agents SDK Email Assistant with Semantic + Episodic + Procedural Memory**

![architecture_pic](https://github.com/panaversity/learn-agentic-ai/blob/main/01_openai_agents/16_memory/01_langmem/00_baseline_email_assistant/img/memory_course_email.png?raw=true)


We previously built an email assistant that:
- Classifies incoming messages (respond, ignore, notify)
- Uses human-in-the-loop to refine the assistant's ability to classify emails
- Drafts responses
- Schedules meetings
- Uses memory to remember details from previous emails

Now, we'll add procedural memory that allows the user to update instructions for using the calendar and email writing tools.

## OpenAI Agents SDK Model Config

In [6]:
from pydantic import BaseModel, Field
from agents import (
    Agent,
    Runner,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from typing import Dict, Any
from typing_extensions import TypedDict, Literal, Annotated


In [7]:
#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=os.environ["GOOGLE_API_KEY"],
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)


## **Setup a Profile, Prompt Instructions and Example Email**

In [8]:
profile = {
    "name": "Junaid",
    "full_name": "Muhammad Junaid Shaukat",
    "user_profile_background": "AI Engineer building personal AI Agents WorkForce.",
}

In [9]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage Junaid tasks efficiently."
}

In [10]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "Quick question about AI documentation",
    "body": """
Hi Junaid,

I was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /agent/auth/refresh
- /agent/auth/validate

Thanks!
Alice""",
}

## Setup LangGraph Store

In [11]:
import asyncio
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langgraph.store.memory import InMemoryStore
from contextlib import asynccontextmanager

store = InMemoryStore(
      index={
          "dims": 768,
          "embed": GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
      }
    )

@asynccontextmanager
async def get_store():
  yield store


Template for formating an example to put in prompt

In [12]:
template = """Email Subject: {subject}
Email From: {from_email}
Email To: {to_email}
Email Content:
```
{content}
```
> Triage Result: {result}"""

# Format list of few shots
def format_few_shot_examples(examples):
    print(examples)
    strs = ["Here are some previous examples:"]
    for eg in examples:
        email_data = eg.value.get("email", {})
        subject = email_data.get("subject", "No Subject")
        to_email = email_data.get("to", "No Recipient")
        from_email = email_data.get("author") or email_data.get("from", "No Sender")
        # Use 'email_thread' if available; otherwise fall back to 'body'
        content = email_data.get("email_thread") or email_data.get("body", "")
        # Truncate the content to a maximum of 400 characters
        content = content[:400]
        result = eg.value.get("label", "No Label")

        strs.append(
            template.format(
                subject=subject,
                to_email=to_email,
                from_email=from_email,
                content=content,
                result=result,
            )
        )
    return "\n\n------------\n\n".join(strs)

## **Prompt Templates**

In [13]:
# Triage system prompt template
triage_system_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few shot examples >
"""





# Triage User Prompt Template
triage_user_prompt_template = """
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}
"""





In [14]:
from pydantic import BaseModel, Field
from typing import Literal

class Email(BaseModel):
    from_: str = Field(alias="from")
    to: str
    subject: str
    body: str

email_model = Email(**email)
print(email_model.model_dump_json(by_alias=True))

{"from":"Alice Smith <alice.smith@company.com>","to":"Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>","subject":"Quick question about AI documentation","body":"\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /agent/auth/refresh\n- /agent/auth/validate\n\nThanks!\nAlice"}


In [15]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

## **A function to create a prompt using f-string**

In [16]:
def create_prompt(template: str, variables: dict[str, any], username: str) -> str:
    """Creates a prompt using an f-string and a dictionary of variables."""
    namespace = (username, )
    result = store.get(namespace, "agent_instructions")
    if result is None:
        store.put(
            namespace,
            "agent_instructions",
            {"prompt": prompt_instructions["agent_instructions"]}
        )
        prompt = prompt_instructions["agent_instructions"]
    else:
        prompt = result.value['prompt']
    try:
        return template.format(instructions=prompt, **variables)
    except KeyError as e:
        return f"Error: Missing variable '{e.args[0]}' in the provided dictionary."

## **Response Agent, Define Tools**

In [17]:
from agents import function_tool

In [18]:
@function_tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"

In [19]:
@function_tool
def schedule_meeting(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"



In [20]:
@function_tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

### **LangMem Memory Management Tools**

### Mem Tools

In [21]:
class UserInfo(BaseModel):
  username: str

namespace_template=("email_assistant", "{username}", "collection")

In [22]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langmem_adapter import LangMemOpenAIAgentToolAdapter

# Initialize the manage memory tool dynamically:
manage_adapter = LangMemOpenAIAgentToolAdapter(
    lambda store, namespace=None: create_manage_memory_tool(namespace=namespace, store=store),
    store_provider=get_store,
    namespace_template=namespace_template
)
manage_memory_tool = manage_adapter.as_tool()

# Initialize the search memory tool dynamically:
search_adapter = LangMemOpenAIAgentToolAdapter(
    lambda store, namespace=None: create_search_memory_tool(namespace=namespace, store=store),
    store_provider=get_store,
    namespace_template=namespace_template
)
search_memory_tool = search_adapter.as_tool()


In [23]:
store.search(("email_assistant", "junaid", "examples"))

[]

## **Response Agent, Define Prompt**

In [24]:
response_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [25]:
tools=[write_email, schedule_meeting, check_calendar_availability, manage_memory_tool, search_memory_tool]

In [26]:
from agents import RunContextWrapper
def dynamic_instructions(
    context: RunContextWrapper[UserInfo], agent: Agent[UserInfo]
) -> str:

    response_system_prompt = create_prompt(response_prompt_template, {
      "full_name": profile["full_name"],
      "name":profile["name"]
    },
      context.context.username
    )
    return response_system_prompt

In [27]:
response_agent = Agent[UserInfo](
    name="Response agent",
    instructions=dynamic_instructions,
    tools=tools
    )


## **Create Triage Agent Flow in Python**

#### Updated triage_router gets ignore, notify and respond rule from store

In [28]:
async def triage_router(email: Email, username: str):

    author = email.from_
    to = email.to
    subject = email.subject
    email_thread = email.body

    namespace = (
        "email_assistant",
        username,
        "examples"
    )

    examples = store.search(
        namespace,
        query=str({"email": email_thread}),
    )
    examples=format_few_shot_examples(examples)

    namespace = (username, )

    result = store.get(namespace, "triage_ignore")
    if result is None:
        store.put(
            namespace,
            "triage_ignore",
            {"prompt": prompt_instructions["triage_rules"]["ignore"]}
        )
        ignore_prompt = prompt_instructions["triage_rules"]["ignore"]
    else:
        ignore_prompt = result.value['prompt']

    result = store.get(namespace, "triage_notify")
    if result is None:
        store.put(
            namespace,
            "triage_notify",
            {"prompt": prompt_instructions["triage_rules"]["notify"]}
        )
        notify_prompt = prompt_instructions["triage_rules"]["notify"]
    else:
        notify_prompt = result.value['prompt']

    result = store.get(namespace, "triage_respond")
    if result is None:
        store.put(
            namespace,
            "triage_respond",
            {"prompt": prompt_instructions["triage_rules"]["respond"]}
        )
        respond_prompt = prompt_instructions["triage_rules"]["respond"]
    else:
        respond_prompt = result.value['prompt']

    system_prompt = create_prompt(triage_system_prompt_template, {
        "full_name": profile["full_name"],
        "name":profile["name"],
        "user_profile_background": profile["user_profile_background"],
        "triage_no": ignore_prompt,
        "triage_notify": notify_prompt,
        "triage_email": respond_prompt,
        "examples": examples,
      },
      username
    )

    user_prompt = triage_user_prompt_template.format(
        author=author,
        to=to,
        subject=subject,
        email_thread=email_thread
    )

    triage_agent = Agent(
        name="Triage Agent",
        instructions=system_prompt,
        output_type=Router
    )

    triage_result = await Runner.run(
        triage_agent,
        user_prompt,
        run_config = config,
        context=UserInfo(username=username)
        )

    print(triage_result.final_output)
    print("Triage History: ", triage_result.to_input_list())

    if triage_result.final_output.classification == "respond":
          print("📧 Classification: RESPOND - This email requires a response")
          response_result = await Runner.run(
              response_agent,
              f"Respond to the email {email.model_dump_json(by_alias=True)}",
              run_config = config,
              context=UserInfo(username=username)
              )
          print(response_result.final_output)
          print("Response History", response_result.to_input_list())
          return response_result.final_output
    elif triage_result.final_output.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
    elif triage_result.final_output.classification == "notify":
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
    else:
        raise ValueError(f"Invalid classification: {triage_result.final_output.classification}")


## **Now Test the Triage and Response Agents Working Together**

In [29]:
email_input = {
    "from": "Alice Jones <alice.jones@bar.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "Quick question about API documentation",
    "body": """Hi Junaid,

Urgent issue - your service is down. Is there a reason why""",
}

typed_email = Email(**email_input)
print(typed_email.model_dump_json(by_alias=True))

{"from":"Alice Jones <alice.jones@bar.com>","to":"Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>","subject":"Quick question about API documentation","body":"Hi Junaid,\n\nUrgent issue - your service is down. Is there a reason why"}


In [30]:
assistant_response = await triage_router(typed_email, "junaid")
assistant_response

[]
reasoning='This is a critical issue (service is down) that requires immediate attention and a direct response from Junaid.' classification='respond'
Triage History:  [{'content': '\nPlease determine how to handle the below email thread:\n\nFrom: Alice Jones <alice.jones@bar.com>\nTo: Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>\nSubject: Quick question about API documentation\nHi Junaid,\n\nUrgent issue - your service is down. Is there a reason why\n', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '{\n  "classification": "respond",\n  "reasoning": "This is a critical issue (service is down) that requires immediate attention and a direct response from Junaid."\n}', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}]
📧 Classification: RESPOND - This email requires a response
Subject: Re: Quick question about API documentation

Hi Alice,

Thanks for reaching out. I'm very sorry to hear about the service disru

"Subject: Re: Quick question about API documentation\n\nHi Alice,\n\nThanks for reaching out. I'm very sorry to hear about the service disruption. I'll look into this immediately and provide you with an update as soon as possible.\n\nBest,\n\nJunaid\n"

In [31]:
import json
type(json.dumps(typed_email.model_dump_json(by_alias=True)))

str

In [32]:
import json
history = [
    {"role": "user", "content": json.dumps(typed_email.model_dump_json(by_alias=True))},
    {"role": "assistant", "content": assistant_response}
    ]

#### and look at current values of long term memory

In [33]:
store.get(("junaid",), "agent_instructions").value['prompt']

'Use these tools when appropriate to help manage Junaid tasks efficiently.'

In [34]:
store.get(("junaid",), "triage_respond").value['prompt']

'Direct questions from team members, meeting requests, critical bug reports'

In [35]:
store.get(("junaid",), "triage_ignore").value['prompt']

'Marketing newsletters, spam emails, mass company announcements'

In [36]:
store.get(("junaid",), "triage_notify").value['prompt']

'Team member out sick, build system notifications, project status updates'

### Now, Use an LLM to update instructions.

In [37]:
from langmem import create_multi_prompt_optimizer

In [38]:
conversations = [
    (
        history,
        "Always sign your emails `Muhammad Junaid`"
    )
]

In [39]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("junaid",), "agent_instructions").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"

    },
    {
        "name": "triage-ignore",
        "prompt": store.get(("junaid",), "triage_ignore").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored"

    },
    {
        "name": "triage-notify",
        "prompt": store.get(("junaid",), "triage_notify").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails the user should be notified of"

    },
    {
        "name": "triage-respond",
        "prompt": store.get(("junaid",), "triage_respond").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be responded to"

    },
]

In [40]:
optimizer = create_multi_prompt_optimizer(
    "google_genai:gemini-2.0-flash",
    kind="prompt_memory",
)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:1410: UserWarning: ChatGoogleGenerativeAI.with_structured_output with dict schema has changed recently to align with behavior of other LangChain chat models. More context: https://github.com/langchain-ai/langchain-google/pull/772
  warnings.warn(


In [41]:
updated = optimizer.invoke(
    {"trajectories": conversations, "prompts": prompts}
)


In [42]:
updated

[{'name': 'main_agent',
  'prompt': 'Use these tools when appropriate to help manage Junaid tasks efficiently. Always sign your emails `Muhammad Junaid`',
  'update_instructions': 'keep the instructions short and to the point',
  'when_to_update': 'Update this prompt whenever there is feedback on how the agent should write emails or schedule events'},
 {'name': 'triage-ignore',
  'prompt': 'Marketing newsletters, spam emails, mass company announcements',
  'update_instructions': 'keep the instructions short and to the point',
  'when_to_update': 'Update this prompt whenever there is feedback on which emails should be ignored'},
 {'name': 'triage-notify',
  'prompt': 'Team member out sick, build system notifications, project status updates',
  'update_instructions': 'keep the instructions short and to the point',
  'when_to_update': 'Update this prompt whenever there is feedback on which emails the user should be notified of'},
 {'name': 'triage-respond',
  'prompt': 'Direct questions f

In [43]:
for i, updated_prompt in enumerate(updated):
    old_prompt = prompts[i]
    if updated_prompt['prompt'] != old_prompt['prompt']:
        name = old_prompt['name']
        print(f"updated {name}")
        if name == "main_agent":
            store.put(
                ("junaid",),
                "agent_instructions",
                {"prompt":updated_prompt['prompt']}
            )
        elif name == "triage-ignore":
            store.put(
                ("junaid",),
                "triage_ignore",
                {"prompt":updated_prompt['prompt']}
            )
        else:
            #raise ValueError
            print(f"Encountered {name}, implement the remaining stores!")

updated main_agent


In [44]:
store.get(("junaid",), "agent_instructions").value['prompt']

'Use these tools when appropriate to help manage Junaid tasks efficiently. Always sign your emails `Muhammad Junaid`'

Now we can use these updated instructions with our agent